In [3]:
import polars as pl
import pandas as pd
import numpy as np
import pyarrow

pd.options.display.float_format = '{:.0f}'.format

## READING CSV DIFFERENCE

In [4]:
df=pl.read_csv("data/movie_data.csv")

In [62]:
df_pd=pd.read_csv("data/movie_data.csv")

## FILTERING DIFFERENCE

In [139]:
df.filter(pl.col("Runtime")>140).shape

(900708, 6)

In [140]:
df_pd[df_pd['Runtime']>140].shape

(900708, 6)

In [137]:
df.filter(pl.col("Genre").is_in(["Documentary","Thriller"])).shape



(2500436, 6)

In [138]:
# df_pd[df_pd['Genre'].isin(["Documentary","Thriller"])]
df_pd.query('Genre in ["Documentary","Thriller"]').shape

(2500436, 6)

## Computation


In [157]:
df.select([pl.col("Genre").unique()])

Genre
str
"""Mystery"""
"""Thriller"""
"""Comedy"""
"""Horror"""
"""Drama"""
"""Romance"""
"""Documentary"""
"""Action"""


In [158]:
df_pd['Genre'].unique()

array(['Romance', 'Drama', 'Action', 'Documentary', 'Thriller', 'Horror',
       'Mystery', 'Comedy'], dtype=object)

## DESCRIBE

In [169]:
df.describe()

describe,Title,Genre,Premiere,Runtime,IMDB Score,Language
str,str,str,str,f64,f64,str
"""count""","""9999999""","""9999999""","""9999999""",9.999999e6,9.999999e6,"""9999999"""
"""null_count""","""0""","""0""","""0""",0.0,0.0,"""0"""
"""mean""",null,null,null,99.495045,3.000622,null
"""std""",null,null,null,28.868762,1.155296,null
"""min""","""A A Baby""","""Action""","""April 01, 2020…",50.0,1.0,"""Chinese """
"""25%""",null,null,null,74.0,2.0,null
"""50%""",null,null,null,99.0,3.0,null
"""75%""",null,null,null,124.0,4.0,null
"""max""","""Yourself Yours…","""Thriller""","""September 30, …",149.0,5.0,"""Spanish """


In [174]:
df_pd.describe()

,Runtime,IMDB Score
count,9999999,9999999
mean,99,3
std,29,1
min,50,1
25%,74,2
50%,99,3
75%,124,4
max,149,5


## CREATION

In [183]:
runtime=df.select(pl.col("Runtime"))

probs = [x for x in range(1,100_000_000)]
percentiles = [x for x in range(1,100_000_000)]

In [184]:
pl.DataFrame(dict(probs=probs, percentiles=percentiles)).shape

(99999999, 2)

In [186]:

pd.DataFrame(dict(probs=probs, percentiles=percentiles)).shape

MemoryError: Unable to allocate 763. MiB for an array with shape (99999999,) and data type object

In [6]:
df.to_pandas()

,Title,Genre,Premiere,Runtime,IMDB Score,Language\r
0,Research Data Security,Romance,"January 10, 2021",87,4,Italian\r
1,Whom Writer Mission,Drama,"June 29, 2022",93,4,Chinese\r
2,Significant During Movement,Action,"February 02, 2020",103,2,English\r
3,Its Gun Suddenly,Romance,"November 15, 2020",139,2,English\r
4,Become Among High,Documentary,"April 26, 2023",134,1,Japanese\r
...,...,...,...,...,...,...
9999994,Happen Management Address,Action,"January 24, 2020",134,2,Chinese\r
9999995,Experience Design Really,Action,"June 11, 2021",104,3,Italian\r
9999996,Concern Effect Eat,Romance,"December 09, 2020",136,3,Hindi\r
9999997,Ahead Poor Later,Action,"January 07, 2022",77,5,Spanish\r


In [ ]:
df.group_by('Genre').agg(pl.count())